In [2]:
import imagej
import os
import scyjava as sj
from scyjava import jimport
import random
import sys
sj.config.add_options('-Xmx12g')

os.environ["JAVA_HOME"] = "/Users/nelsschimek/mambaforge/envs/i2k-2022-pyimagej/"

In [3]:
# initialize ImageJ
ij = imagej.init('/Applications/Fiji.app/')
print(f"ImageJ version: {ij.getVersion()}")

ImageJ version: 2.9.0/1.53t


In [20]:
Runtime = jimport('java.lang.Runtime')

In [21]:
print(Runtime.getRuntime().maxMemory() // (2**20), " MB available to java")

10923  MB available to java


In [5]:
image_path = "/Users/nelsschimek/Documents/nancelab/diff_classifier/notebooks/development/MPT_Data/TIFs/P10F_NT_10DIV_40nm_slice_2_midbrain_vid_2.tif"
image = ij.io().open(image_path)

In [22]:
# def dump_info(image):
#     """A handy function to print details of an image object."""
#     name = image.name if hasattr(image, 'name') else None # xarray
#     if name is None and hasattr(image, 'getName'): name = image.getName() # Dataset
#     if name is None and hasattr(image, 'getTitle'): name = image.getTitle() # ImagePlus
#     print(f" name: {name or 'N/A'}")
#     print(f" type: {type(image)}")
#     print(f"dtype: {image.dtype if hasattr(image, 'dtype') else 'N/A'}")
#     print(f"shape: {image.shape}")
#     print(f" dims: {image.dims if hasattr(image, 'dims') else 'N/A'}")

# if not 'image' in locals(): print("I cannot find any variable called embryo. Did you write `embryo = ...` above?")
# elif str(type(image)) != "<java class 'net.imagej.DefaultDataset'>": print("The embryo variable does not contain an ImageJ2 Dataset. Did you use the `ij.io().open` function?")
# else: dump_info(image)

In [4]:
Model = sj.jimport('fiji.plugin.trackmate.Model')
Settings = sj.jimport('fiji.plugin.trackmate.Settings')
TrackMate = sj.jimport('fiji.plugin.trackmate.TrackMate')
FeatureFilter = sj.jimport('fiji.plugin.trackmate.features.FeatureFilter')
LAPUtils =  sj.jimport('fiji.plugin.trackmate.tracking.jaqaman.LAPUtils')
SelectionModel = sj.jimport('fiji.plugin.trackmate.SelectionModel')
Logger = sj.jimport('fiji.plugin.trackmate.Logger')

DisplaySettingsIO = sj.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
HyperStackDisplayer = sj.jimport('fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer')

In [5]:
LogDetectorFactory = sj.jimport('fiji.plugin.trackmate.detection.LogDetectorFactory')

In [6]:
SparseLAPTrackerFactory = sj.jimport('fiji.plugin.trackmate.tracking.jaqaman.SparseLAPTrackerFactory')

In [7]:
model = Model()

In [8]:
model.setLogger(Logger.IJ_LOGGER)

In [34]:

im = ij.io().open('https://fiji.sc/samples/FakeTracks.tif')


In [ ]:
imp = ij.py.to_imageplus(im)

In [11]:
settings = Settings()

In [12]:
settings.detectorFactory = LogDetectorFactory()
settings.detectorSettings = {
    'DO_SUBPIXEL_LOCALIZATION' : True,
    'RADIUS' : 2.5,
    'TARGET_CHANNEL' : ij.py.to_java(1),
    'THRESHOLD' : 0.,
    'DO_MEDIAN_FILTERING' : False,
}  
filter1 = FeatureFilter('QUALITY', 30, True)
settings.addSpotFilter(filter1)


In [13]:
# Configure tracker
settings.trackerFactory = SparseLAPTrackerFactory()
settings.trackerSettings = LAPUtils.getDefaultSegmentSettingsMap()
settings.trackerSettings['LINKING_MAX_DISTANCE'] = 10.0
settings.trackerSettings['GAP_CLOSING_MAX_DISTANCE'] = 10.0
settings.trackerSettings['MAX_FRAME_GAP'] = ij.py.to_java(3)

In [14]:
trackmate = TrackMate(model, settings)

In [16]:
TrackMate.__dict__

mappingproxy({'PLUGIN_NAME_STR': <java field 'PLUGIN_NAME_STR' of 'fiji.plugin.trackmate.TrackMate'>,
              'PLUGIN_NAME_VERSION': <java field 'PLUGIN_NAME_VERSION' of 'fiji.plugin.trackmate.TrackMate'>,
              'cancel': <java method 'cancel' of 'fiji.plugin.trackmate.TrackMate'>,
              'checkInput': <java method 'checkInput' of 'fiji.plugin.trackmate.TrackMate'>,
              'computeEdgeFeatures': <java method 'computeEdgeFeatures' of 'fiji.plugin.trackmate.TrackMate'>,
              'computeSpotFeatures': <java method 'computeSpotFeatures' of 'fiji.plugin.trackmate.TrackMate'>,
              'computeTrackFeatures': <java method 'computeTrackFeatures' of 'fiji.plugin.trackmate.TrackMate'>,
              'execDetection': <java method 'execDetection' of 'fiji.plugin.trackmate.TrackMate'>,
              'execInitialSpotFiltering': <java method 'execInitialSpotFiltering' of 'fiji.plugin.trackmate.TrackMate'>,
              'execSpotFiltering': <java method 'execSp

In [39]:
trackmate.execDetection()

Starting detection process using 8 threads.



[java.lang.Enum.toString] Starting detection process using 8 threads.
[java.lang.Enum.toString] Starting detection process using 8 threads.
[java.lang.Enum.toString] Starting detection process using 8 threads.
[java.lang.Enum.toString] 
[java.lang.Enum.toString] 
[java.lang.Enum.toString] 


java.lang.NullPointerException: java.lang.NullPointerException

In [152]:
ok = trackmate.checkInput()
if not ok:
    sys.exit(str(trackmate.getErrorMessage()))

In [5]:
from scyjava import jimport
Runtime = jimport('java.lang.Runtime')
def java_mem():
    rt = Runtime.getRuntime()
    mem_max = rt.maxMemory()
    mem_used = rt.totalMemory() - rt.freeMemory()
    return '{} of {} MB ({}%)'.format(int(mem_used)/2**20, int(mem_max/2**20), int(100*mem_used/mem_max))

java_mem()

'270.4136734008789 of 10923 MB (2%)'

In [151]:


ok = trackmate.process()
if not ok:
    sys.exit(str(trackmate.getErrorMessage()))

Starting detection process using 8 threads.



[java.lang.Enum.toString] Starting detection process using 8 threads.
[java.lang.Enum.toString] 


java.lang.OutOfMemoryError: java.lang.OutOfMemoryError: GC overhead limit exceeded

In [139]:
model.getLogger().log('Found ' + str(model.getTrackModel().nTracks(True)) + ' tracks.')


Found 9 tracks.


[java.lang.Enum.toString] Found 9 tracks.[java.lang.Enum.toString] 


In [143]:
# A selection.
sm = SelectionModel( model )

# Read the default display settings.
ds = DisplaySettingsIO.readUserDefault()

# The viewer.
displayer =  HyperStackDisplayer( model, sm, imp, ds ) 
#displayer.render()

# The feature model, that stores edge and track features.
fm = model.getFeatureModel()

In [144]:
# Iterate over all the tracks that are visible.
for id in model.getTrackModel().trackIDs(True):

    # Fetch the track feature from the feature model.
    v = fm.getTrackFeature(id, 'TRACK_MEAN_SPEED')
    model.getLogger().log('')
    model.getLogger().log('Track ' + str(id) + ': mean velocity = ' + str(v) + ' ' + model.getSpaceUnits() + '/' + model.getTimeUnits())

	# Get all the spots of the current track.
    track = model.getTrackModel().trackSpots(id)
    for spot in track:
        sid = spot.ID()
        # Fetch spot features directly from spot.
        # Note that for spots the feature values are not stored in the FeatureModel
        # object, but in the Spot object directly. This is an exception; for tracks
        # and edges, you have to query the feature model.
        x=spot.getFeature('POSITION_X')
        y=spot.getFeature('POSITION_Y')
        t=spot.getFeature('FRAME')
        q=spot.getFeature('QUALITY')
        snr=spot.getFeature('SNR_CH1')
        mean=spot.getFeature('MEAN_INTENSITY_CH1')
        model.getLogger().log('\tspot ID = ' + str(sid) + ': x='+str(x)+', y='+str(y)+', t='+str(t)+', q='+str(q) + ', snr='+str(snr) + ', mean = ' + str(mean))

[java.lang.Enum.toString] 


TypeError: can only concatenate str (not "java.lang.String") to str

In [145]:
fm

<java object 'fiji.plugin.trackmate.FeatureModel'>